<a href="https://colab.research.google.com/github/ShadowMonarch001/DreamCraft-AI-Image-Forge/blob/main/DreamCraft_AI_Image_Forge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install diffusers==0.10.2 transformers scipy ftfy accelerate  # To run Stable Diffusion
!pip install pyngrok==4.1.1  # To register our ngrok token
!pip install flask_ngrok  # So we can make flask app port thru ngrok

In [46]:
!ngrok authtoken <your_auth_token>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [47]:
import os
if not os.path.isdir('/content/app'):
  os.mkdir("/content/app")
os.chdir("/content/app")

In [48]:
if not os.path.isdir('/content/app/static'):
  os.mkdir("./static")
os.chdir("./static")

css = """
/* CSS */
body {
    margin: 0;
    padding: 0;
    background: linear-gradient(120deg, #1a1a1a, #333333);
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    color: white;
}

.container {
    display: flex;
    flex-direction: column;
    align-items: center;
    justify-content: center;
    height: 100vh;
}

.title {
    font-size: 3em;
    font-weight: bold;
    color: #ff3333;
    text-transform: uppercase;
    margin-bottom: 1em;
    text-shadow: 2px 2px 4px rgba(0,0,0,0.5);
    letter-spacing: 2px;
    text-align: center;
}

.image-container {
    position: relative;
    width: 80%;
    max-width: 600px;
    margin-bottom: 2em;
    border-radius: 10px;
    overflow: hidden;
    box-shadow: 0px 0px 20px rgba(255, 255, 255, 0.5);
}

img {
    width: 100%;
    height: auto;
}

.textbox {
    width: 80%;
    max-width: 600px;
    margin-bottom: 2em;
    display: flex;
    flex-direction: column;
    align-items: center;
}

input[type="text"] {
    width: 100%;
    padding: 1em;
    font-size: 1.2em;
    border: none;
    border-radius: 5px;
    background-color: rgba(255, 255, 255, 0.1);
    color: white;
    outline: none;
    margin-bottom: 10px;
}

input[type="text"]::placeholder {
    color: rgba(255, 255, 255, 0.5);
}

button#generateButton {
    padding: 1em 2em;
    background-color: #ff3333;
    color: white;
    border: none;
    border-radius: 5px;
    font-size: 1.2em;
    cursor: pointer;
    transition: background-color 0.3s;
    text-transform: uppercase;
    letter-spacing: 2px;
    box-shadow: 0px 4px 10px rgba(0,0,0,0.2);
    align-self: flex-start;
}

button#generateButton:hover {
    background-color: #ff6666;
}

"""

with open("styles.css", 'w') as f:
  f.write(css)

os.chdir("..")

In [49]:
html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>DreamCraft:AI Image Forge</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
    <link rel="stylesheet" href="styles.css">
</head>
<body>
    <div class="container">
        <h1 class="title">DreamCraft:AI Image Forge</h1>
        <div class="image-container">
            <img id="generatedImage" src="{{ generated_image|default("https://www.ghacks.net/wp-content/uploads/2023/08/AI-generated-art-copyright.webp?format=512w", true) }}" class="mainImg" alt="Generated Image">
        </div>
        <div class="textbox">
            <form action="\submit-prompt" method="post" id="submission-form">
                <input type="text" id="prompt" name="prompt-input" placeholder="Enter image description">
                <button id="generateButton" type="submit">Generate</button>
            </form>
        </div>
    </div>
</body>
</html>


"""

with open("index.html", 'w') as f:
  f.write(html)


In [50]:
import os
os.chdir("/content/app")

In [51]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, send_file, request

import torch
from diffusers import StableDiffusionPipeline

import base64
from io import BytesIO

# Load model
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", revision="fp16", torch_dtype=torch.float16)
pipe.to("cuda")

# Start flask app and set to ngrok
app = Flask(__name__, template_folder='.') # period is because default templates folder is /templates
run_with_ngrok(app)

@app.route('/')
def initial():
  return render_template('index.html')

@app.route('/submit-prompt', methods=['POST'])
def generate_image():
  prompt = request.form['prompt-input']
  print(f"Generating an image of {prompt}")

  image = pipe(prompt).images[0]
  print("Image generated! Converting image ...")

  buffered = BytesIO()
  image.save(buffered, format="PNG")
  img_str = base64.b64encode(buffered.getvalue())
  b = "data:image/png;base64," + str(img_str)[2:-1]

  print("Sending image ...")
  return render_template('index.html', generated_image=b)

app.run()

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://33d7-34-168-202-231.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Feb/2024 11:36:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Feb/2024 11:36:41] "GET /styles.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Feb/2024 11:36:41] "GET /static/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Feb/2024 11:36:42] "GET /favicon.ico HTTP/1.1" 404 -
